In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('/Users/brianoktavec/MSC550/Assignment_4/Auto.csv')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,name
0,18.0,8,307.0,130,3504,12.0,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,ford torino


In [3]:
# 1

In [4]:
data = {'model1':[0.757,2178,2214], 'model2':[0.704, 2252,2276], 'model3':[0.789, -336.6,-312.8], 'model4':[0.789, -336.6,-312.8], 'model5':[0.776, 148.9,172.7]}
df2 = pd.DataFrame(data, index =['Adj.R^2', 'AIC', 'BIC'])  

In [5]:
print(df2)

           model1    model2   model3   model4   model5
Adj.R^2     0.757     0.704    0.789    0.789    0.776
AIC      2178.000  2252.000 -336.600 -336.600  148.900
BIC      2214.000  2276.000 -312.800 -312.800  172.700


In [15]:
# 2

In [6]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
mpg_log = np.log(df.iloc[:,0:1])
# mpg_log = pd.DataFrame(mpg_log[:,0:], columns = ['mpg','mpg_log'])
mpg_log = mpg_log.rename(columns={"mpg": "Log_mpg"})
mpg = df.mpg
cylinders = poly.fit_transform(df.iloc[:,1:2])
cylinders = pd.DataFrame(cylinders[:,1:], columns = ['cylinders_1','cylinders_2'])
cylinders_log = np.log(df.iloc[:,1:2])
cylinders_log = cylinders_log.rename(columns={"cylinders": "Log_cylinders"})
displacement = poly.fit_transform(df.iloc[:,2:3])
displacement = pd.DataFrame(displacement[:,1:], columns = ['displacement_1','displacement_2'])
displacement_log = np.log(df.iloc[:,2:3])
displacement_log = displacement_log.rename(columns={"displacement": "Log_displacement"})
horsepower_log = np.log(df.iloc[:,3:4])
horsepower_log = horsepower_log.rename(columns={"horsepower": "Log_horsepower"})
horsepower = poly.fit_transform(df.iloc[:,3:4])
horsepower = pd.DataFrame(horsepower[:,1:], columns = ['horsepower_1','horsepower_2'])
acceleration = poly.fit_transform(df.iloc[:,4:5])
acceleration = pd.DataFrame(acceleration[:,1:], columns = ['acceleration_1','acceleration_2'])
acceleration_log = np.log(df.iloc[:,4:5])
acceleration_log = acceleration_log.rename(columns={"acceleration": "Log_acceleration"})
Z, fit_lambda = stats.boxcox(df.mpg)
Z = pd.DataFrame(Z, columns = ['Z'])
df_new = pd.concat([cylinders, cylinders_log, displacement, displacement_log, horsepower, horsepower_log, acceleration, acceleration_log, mpg, mpg_log, Z, df.iloc[:,15:]], axis=1)
df_new

,cylinders_1,cylinders_2,Log_cylinders,displacement_1,displacement_2,Log_displacement,horsepower_1,horsepower_2,Log_horsepower,acceleration_1,acceleration_2,weight,mpg,Log_mpg,Z
0,8.0,64.0,2.079442,307.0,94249.0,5.726848,130.0,16900.0,4.867534,3504.0,12278016.0,8.161660,18.0,2.890372,3.859148
1,8.0,64.0,2.079442,350.0,122500.0,5.857933,165.0,27225.0,5.105945,3693.0,13638249.0,8.214194,15.0,2.708050,3.547736
2,8.0,64.0,2.079442,318.0,101124.0,5.762051,150.0,22500.0,5.010635,3436.0,11806096.0,8.142063,18.0,2.890372,3.859148
3,8.0,64.0,2.079442,304.0,92416.0,5.717028,150.0,22500.0,5.010635,3433.0,11785489.0,8.141190,16.0,2.772589,3.656731
4,8.0,64.0,2.079442,302.0,91204.0,5.710427,140.0,19600.0,4.941642,3449.0,11895601.0,8.145840,17.0,2.833213,3.760349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,4.0,16.0,1.386294,140.0,19600.0,4.941642,86.0,7396.0,4.454347,2790.0,7784100.0,7.933797,27.0,3.295837,4.591882
388,4.0,16.0,1.386294,97.0,9409.0,4.574711,52.0,2704.0,3.951244,2130.0,4536900.0,7.663877,44.0,3.784190,5.553254
389,4.0,16.0,1.386294,135.0,18225.0,4.905275,84.0,7056.0,4.430817,2295.0,5267025.0,7.738488,32.0,3.465736,4.916211
390,4.0,16.0,1.386294,120.0,14400.0,4.787492,79.0,6241.0,4.369448,2625.0,6890625.0,7.872836,28.0,3.332205,4.660422


In [7]:
from sklearn.model_selection import train_test_split
error1 = []
error2 = []
error3 = []
error4 = []
error5 = []
for i in range(1, 100):
    x = df_new.iloc[:, :15]
    y = df_new.iloc[:, 10:]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
    
    ols = sm.OLS(y_train[["mpg"]], sm.add_constant(x_train[["cylinders_2","displacement_2","horsepower_2","acceleration_2","weight"]]))
    lm1 = ols.fit()
    y_pred1 = lm1.predict(sm.add_constant(x_test[["cylinders_2","displacement_2","horsepower_2","acceleration_2","weight"]]))
    error1.append(mean_squared_error(y_pred1,y_test[["mpg"]]))
    
    ols = sm.OLS(y_train[["mpg"]], sm.add_constant(x_train[["Log_cylinders","Log_displacement","Log_horsepower","weight"]]))
    lm2 = ols.fit()
    y_pred2 = lm2.predict(sm.add_constant(x_test[["Log_cylinders","Log_displacement","Log_horsepower","weight"]]))
    error2.append(mean_squared_error(y_pred2,y_test[["mpg"]]))    
    
    ols = sm.OLS(y_train[["Log_mpg"]], sm.add_constant(x_train[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))
    lm3 = ols.fit()
    y_pred3 = lm3.predict(sm.add_constant(x_test[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))
    error3.append(mean_squared_error(y_pred3,y_test[["Log_mpg"]]))  
    
    ols = sm.OLS(y_train[["Log_mpg"]], sm.add_constant(x_train[["Log_cylinders","Log_displacement","Log_horsepower","weight"]]))
    lm4 = ols.fit()
    y_pred4 = np.exp(lm4.predict(sm.add_constant(x_test[["Log_cylinders","Log_displacement","Log_horsepower","weight"]])))
    error4.append(mean_squared_error(y_pred4,y_test[["Log_mpg"]]))  
    
    ols = sm.OLS(y_train[["Z"]], sm.add_constant(x_train[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))
    lm5 = ols.fit()
    y_pred5 = (lm5.predict(sm.add_constant(x_test[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))*fit_lambda+1)**(1/fit_lambda)
    error5.append(mean_squared_error(y_pred5,y_test[["Z"]]))      


In [14]:
average1 = sum(error1)/len(error1)
average2 = sum(error2)/len(error2)
average3 = sum(error3)/len(error3)
average4 = sum(error4)/len(error4)
average5 = sum(error5)/len(error5)

print("Average MSE of model 1 is:", average1)
print("Average MSE of model 2 is:", average2)
print("Average MSE of model 3 is:", average3)
print("Average MSE of model 4 is:", average4)
print("Average MSE of model 5 is:", average5)

Average MSE of model 1 is: 16.940252265019435
Average MSE of model 2 is: 16.52324701613996
Average MSE of model 3 is: 0.024658642371471088
Average MSE of model 4 is: 447.9960072808513
Average MSE of model 5 is: 399.306571466452


In [16]:
# 3

In [25]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=10, random_state=1, shuffle=True)
x = df_new.iloc[:, :12]
y = df_new.iloc[:, 12:]
error11 = []
error22 = []
error33 = []
error44 = []
error55 = []
for train_index, test_index in cv.split(df_new):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    ols = sm.OLS(y_train[["mpg"]], sm.add_constant(x_train[["cylinders_2","displacement_2","horsepower_2","acceleration_2","weight"]]))
    lm1 = ols.fit()
    y_pred1 = lm1.predict(sm.add_constant(x_test[["cylinders_2","displacement_2","horsepower_2","acceleration_2","weight"]]))
    error11.append(mean_squared_error(y_pred1,y_test[["mpg"]]))
    
    ols = sm.OLS(y_train[["mpg"]], sm.add_constant(x_train[["Log_cylinders","Log_displacement","Log_horsepower","weight"]]))
    lm2 = ols.fit()
    y_pred2 = lm2.predict(sm.add_constant(x_test[["Log_cylinders","Log_displacement","Log_horsepower","weight"]]))
    error22.append(mean_squared_error(y_pred2,y_test[["mpg"]]))    
    
    ols = sm.OLS(y_train[["Log_mpg"]], sm.add_constant(x_train[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))
    lm3 = ols.fit()
    y_pred3 = lm3.predict(sm.add_constant(x_test[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))
    error33.append(mean_squared_error(y_pred3,y_test[["Log_mpg"]]))  
    
    ols = sm.OLS(y_train[["Log_mpg"]], sm.add_constant(x_train[["Log_cylinders","Log_displacement","Log_horsepower","weight"]]))
    lm4 = ols.fit()
    y_pred4 = np.exp(lm4.predict(sm.add_constant(x_test[["Log_cylinders","Log_displacement","Log_horsepower","weight"]])))
    error44.append(mean_squared_error(y_pred4,y_test[["Log_mpg"]]))  
    
    ols = sm.OLS(y_train[["Z"]], sm.add_constant(x_train[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))
    lm5 = ols.fit()
    y_pred5 = (lm5.predict(sm.add_constant(x_test[["cylinders_1","displacement_1","horsepower_1","acceleration_1","weight"]]))*fit_lambda+1)**(1/fit_lambda)
    error55.append(mean_squared_error(y_pred5,y_test[["Z"]]))      


In [27]:
average11 = sum(error11)/len(error11)
average22 = sum(error22)/len(error22)
average33 = sum(error33)/len(error33)
average44 = sum(error44)/len(error44)
average55 = sum(error55)/len(error55)

print("Average cross validation of model 1 is:", average11)
print("Average cross validation of model 2 is:", average22)
print("Average cross validation of model 3 is:", average33)
print("Average cross validation of model 4 is:", average44)
print("Average cross validation of model 5 is:", average55)

Average cross validation of model 1 is: 16.814940877306462
Average cross validation of model 2 is: 16.181848983076616
Average cross validation of model 3 is: 0.02441555820199265
Average cross validation of model 4 is: 444.11524426897165
Average cross validation of model 5 is: 396.38638543377203
